In [7]:
import os

# Get the current folder
current_folder = os.getcwd()

# Print the current folder
print(current_folder)

D:\Giang\Code\GNN-Resource-Management\NewDir


In [8]:
# Set the desired directory path
new_directory = 'D:/Giang/Code/GNN-Resource-Management/NewDir'

# Change the current directory
os.chdir(new_directory)

In [10]:
import scipy.io

# Load .mat file
data = scipy.io.loadmat('Allocation_8Ids_3APs.mat')

# Access the variables
Mu = data['Mu']
Tau = data['Tau']
Power = data['Power']
ChannelAll = data['ChannelAll']

In [10]:
Mu

array([[0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [11]:
Tau

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [12]:
Power

array([[7.19693747e+00, 1.48189778e+01, 5.76762579e+00, 6.77059628e-06,
        6.19523472e+00, 1.08581442e-08, 1.32132602e+01, 4.76384006e+00]])

In [13]:
ChannelAll

array([[5.44011078e-07, 2.64449106e-05, 2.98598317e-06],
       [2.15862365e-07, 6.78765907e-05, 1.19251445e-06],
       [8.11006265e-07, 5.95050178e-06, 7.64526091e-06],
       [2.56285471e-07, 8.60366171e-07, 1.31314600e-06],
       [1.26782153e-06, 4.70406099e-07, 6.19311804e-07],
       [1.82246142e-06, 2.20294781e-07, 2.47949891e-06],
       [4.20545451e-06, 2.24559726e-07, 1.26098801e-06],
       [4.05121889e-06, 9.69237742e-07, 7.51929511e-07]])

# Test WSN

In [14]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn.conv import MessagePassing
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, Sigmoid, BatchNorm1d as BN

from reImplement import GCNet
from setup_arguments import setup_args


In [16]:
def generate_channels_wsn(num_ap, num_user, num_samples, var_noise=1.0, radius=1):
    # print("Generating Data for training and testing")

    # if num_ap != 1:
    #     raise Exception("Can not generate data for training and testing with more than 1 base station")
    # generate position
    dist_mat = []
    position = []
    index_user = np.tile(np.arange(num_user), (num_ap, 1))
    index_ap = np.tile(np.arange(num_ap).reshape(-1, 1), (1, num_user))

    index = np.array([index_user, index_ap])

    # Calculate channel
    CH = 1 / np.sqrt(2) * (np.random.randn(num_samples, 1, num_user)
                           + 1j * np.random.randn(num_samples, 1, num_user))

    if radius == 0:
        Hs = abs(CH)
    else:
        for each_sample in range(num_samples):
            pos = []
            pos_BS = []

            for i in range(num_ap):
                r = radius * (np.random.rand())
                theta = np.random.rand() * 2 * np.pi
                pos_BS.append([r * np.sin(theta), r * np.cos(theta)])
                pos.append([r * np.sin(theta), r * np.cos(theta)])
            pos_user = []

            for i in range(num_user):
                r = 0.5 * radius + 0.5 * radius * np.random.rand()
                theta = np.random.rand() * 2 * np.pi
                pos_user.append([r * np.sin(theta), r * np.cos(theta)])
                pos.append([r * np.sin(theta), r * np.cos(theta)])

            pos = np.array(pos)
            pos_BS = np.array(pos_BS)
            dist_matrix = distance_matrix(pos_BS, pos_user)
            # dist_matrixp = distance_matrix(pos[1:], pos[1:])
            dist_mat.append(dist_matrix)
            position.append(pos)

        dist_mat = np.array(dist_mat)
        position = np.array(position)

        # Calculate Free space pathloss
        # f = 2e9
        # c = 3e8
        # FSPL_old = 1 / ((4 * np.pi * f * dist_mat / c) ** 2)
        FSPL = - (120.9 + 37.6 * np.log10(dist_mat/1000))
        FSPL = 10 ** (FSPL / 10)

        # print(f'FSPL_old:{FSPL_old.sum()}')
        # print(f'FSPL_new:{FSPL.sum()}')
        Hs = abs(CH * FSPL)

    adj = adj_matrix(num_user * num_ap)

    Hs, noise = normalize_matrix(Hs, var_noise)

    return Hs, noise, position, adj, index

def adj_matrix(num_nodes):
    adj = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if not (i == j):
                adj.append([i, j])
    return np.array(adj)



In [18]:
def graph_build(channel_matrix, index_matrix):
    num_user, num_ap = channel_matrix.shape
    adjacency_matrix = adj_matrix(num_user * num_ap)

    index_user = np.reshape(index_matrix[0], (-1, 1))
    index_ap = np.reshape(index_matrix[1], (-1, 1))

    x1 = np.reshape(channel_matrix, (-1, 1))
#     x2 = np.ones((num_user * num_ap, 1)) # power max here, for each?
    x2 = np.zeros((num_user * num_ap, 1)) # ap selection
    x3 = np.zeros((num_user * num_ap, 1))
    x = np.concatenate((x1, x2, x3),axis=1)

    edge_index = adjacency_matrix
    edge_attr = []

    for each_interference in adjacency_matrix:
        tx = each_interference[0]
        rx = each_interference[1]

        tmp = [channel_matrix[index_ap[rx][0]][index_user[tx][0]]]
#         tmp = [
#             [channel_matrix[index_ap[rx][0]][index_user[tx][0]]],
#             [channel_matrix[index_ap[tx][0]][index_user[rx][0]]]
#         ]
        edge_attr.append(tmp)

    # y = np.expand_dims(channel_matrix, axis=0)
    # pos = np.expand_dims(weights_matrix, axis=0)

    data = Data(x=torch.tensor(x, dtype=torch.float),
                edge_index=torch.tensor(edge_index, dtype=torch.long).t().contiguous(),
                edge_attr=torch.tensor(edge_attr, dtype=torch.float),
                # y=torch.tensor(y, dtype=torch.float),
                # pos=torch.tensor(pos, dtype=torch.float)
                )
    return data

def build_all_data(channel_matrices, index_mtx):
    num_sample = channel_matrices.shape[0]
    data_list = []
    for i in range(num_sample):
        data = graph_build(channel_matrices[i], index_mtx)
        data_list.append(data)

    return data_list

def data_rate_calc(data, out, num_ap, num_user, noise_matrix, p_max, train = True, isLog=False):
    G = torch.reshape(out[:, 0], (-1, num_ap, num_user))  #/ noise
    ap_select = torch.reshape(out[:, 2], (-1, num_ap, num_user))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # how to get channel from data and output
    P = torch.reshape(out[:, 2], (-1, num_ap, num_user)) * p_max
    P = torch.mul(P,ap_select)
    desired_signal = torch.sum(torch.mul(P,G), dim=2).unsqueeze(-1)
    P_UE = torch.sum(P, dim=1).unsqueeze(-1)
    all_received_signal = torch.matmul(G, P_UE)
    new_noise = torch.from_numpy(noise_matrix).to(device)
    interference = all_received_signal - desired_signal + new_noise
    rate = torch.log(1 + torch.div(desired_signal, interference))
    sum_rate = torch.mean(torch.sum(rate, 1))
    mean_power = torch.mean(torch.sum(P_UE, 1))

    if(isLog):
      print(f'Channel Coefficient: {G}')
      print(f'Power: {P}')
      print(f'desired_signal: {desired_signal}')
      print(f'P_UE: {P_UE}')
      print(f'all_received_signal: {all_received_signal}')
      print(f'interference: {interference}')

    if train:
        return torch.neg(sum_rate/mean_power)
    else:
        return sum_rate/mean_power



In [ ]:
K = 3  # number of APs
N = 5  # number of nodes
R = 10  # radius

num_train = 2  # number of training samples
num_test = 4  # number of test samples

reg = 1e-2
pmax = 1
var_db = 10
var = 1 / 10 ** (var_db / 10)
var_noise = 10e-11

power_threshold = 2.0

X_train, noise_train, pos_train, adj_train, index_train = generate_channels_wsn(K, N, num_train, var_noise, R)
X_test, noise_test, pos_test, adj_test, index_test = generate_channels_wsn(K + 1, N + 10, num_test, var_noise, R)





In [ ]:
# Preparing Data in to graph structured for model
train_data_list = build_all_data(X_train, index_train)
test_data_list = build_all_data(X_test, index_test)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)

train_loader = DataLoader(train_data_list, batch_size=10, shuffle=True, num_workers=1)
test_loader = DataLoader(test_data_list, batch_size=10, shuffle=False, num_workers=1)

In [ ]:
training_loss = []
testing_loss = []
# Training and Testing model
for epoch in range(1, 100):
    total_loss = 0
    for each_data in train_loader:
        data = each_data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = data_rate_calc(data, out, K, N, noise_train, power_threshold, train=True)
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()

    train_loss = total_loss / num_train

    model.eval()
    total_loss = 0
    for each_data in test_loader:
        data = each_data.to(device)
        out = model(data)
        loss = data_rate_calc(data, out, K + 1, N + 10, noise_test, power_threshold,  train=False)
        total_loss += loss.item() * data.num_graphs

    test_loss = total_loss / num_test

    training_loss.append(train_loss)
    testing_loss.append(test_loss)
    if (epoch % 8 == 1):
        print('Epoch {:03d}, Train Loss: {:.4f}, Val Loss: {:.4f}'.format(
            epoch, train_loss, test_loss))
    scheduler.step()

# Edge Convolution

In [2]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


In [3]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [9]:
print(f'dataset.num_features: {dataset.num_features}')
print(f'dataset.num_classes : {dataset.num_classes }')
print(f'dataset.num_features: {dataset.num_nodes}')
print(f'dataset.num_classes : {dataset.num_classes }')

dataset.num_features: 1433
dataset.num_classes : 7


AttributeError: 'Planetoid' object has no attribute 'num_nodes'

# Heterogenous Data

In [12]:
import torch
from torch_geometric.data import HeteroData

In [13]:


data = HeteroData()

num_users = 5
num_users_features = 3

num_aps = 3
num_aps_features = 3

# Create two node types "paper" and "author" holding a feature matrix:
data['user'].x = torch.randn(num_users, num_users_features) # features of user_node
data['ap'].x = torch.randn(num_aps, num_aps_features) # features of user_node

# Create edge types and building the
# graph connectivity:
data['user', 'up', 'ap'].edge_index = ...  # [2, num_edges]
data['ap', 'down', 'user'].edge_index = ...  # [2, num_edges]



In [15]:
data['user'].num_nodes

5

In [18]:
data['ap'].num_nodes

3

In [17]:
data['user', 'up', 'ap'].num_edges

0

In [19]:
data['ap', 'user'].num_edges

0

In [20]:
data

HeteroData(
  user={ x=[5, 3] },
  ap={ x=[3, 3] },
  paper={},
  (user, up, ap)={ edge_index=Ellipsis },
  (ap, down, user)={ edge_index=Ellipsis }
)

In [25]:
subset_dict = {
    'ap': torch.tensor([0, 1]),
    'user': torch.tensor([0, 2]),
}

In [27]:
print(data.collect('x'))

{'user': tensor([[ 1.6823,  0.8296,  1.7004],
        [-0.5770, -1.0360, -1.2604],
        [-0.0759,  1.3773, -0.9175],
        [ 0.2248,  0.8405,  1.3123],
        [ 0.9722, -0.1851,  0.0476]]), 'ap': tensor([[-0.9269,  1.0285,  0.5206],
        [-1.2664, -0.6786, -0.1242],
        [ 0.8528, -2.1334, -0.3032]])}


In [1]:
import torch
import numpy as np
import scipy  # for testing
from torch_geometric.data import HeteroData

from WSN_GNN import generate_channels_wsn

In [31]:
num_users = 5
num_users_features = 3

num_aps = 3
num_aps_features = 3


def convert_to_hetero_data(channel_matrices):
    graph_list = []
    num_sam, num_aps, num_users = channel_matrices.shape
    for i in range(num_sam):
        user_feat = torch.zeros(num_users, num_users_features)  # features of user_node
        ap_feat = torch.zeros(num_aps, num_aps_features)  # features of user_node
        edge_feat_uplink = channel_matrices[i, :, :].reshape(-1, 1)
        edge_feat_downlink = channel_matrices[i, :, :].reshape(-1, 1)
        graph = HeteroData({
            'user': {'x': user_feat},
            'ap': {'x': ap_feat}
        })
        # Create edge types and building the graph connectivity:
        graph['user', 'uplink', 'ap'].edge_attr  = torch.tensor(edge_feat_uplink, dtype=torch.float)
#         graph['ap', 'downlink', 'user'].edge_attr  = torch.tensor(edge_feat_downlink, dtype=torch.float)
        graph_list.append(graph)
    return graph_list

In [43]:
K = 3  # number of APs
N = 5  # number of nodes
R = 10  # radius

num_train = 400  # number of training samples
num_test = 4  # number of test samples

reg = 1e-2
pmax = 1
var_db = 10
var = 1 / 10 ** (var_db / 10)
var_noise = 10e-11

power_threshold = 2.0

X_train, noise_train, pos_train, adj_train, index_train = generate_channels_wsn(K, N, num_train, var_noise, R)
X_test, noise_test, pos_test, adj_test, index_test = generate_channels_wsn(K + 1, N + 10, num_test, var_noise, R)

train_data = convert_to_hetero_data(X_train)
print(train_data[0].edge_stores)

[{'edge_attr': tensor([[0.0994],
        [0.0209],
        [0.1523],
        [0.1551],
        [0.3332],
        [0.0233],
        [0.0112],
        [0.3298],
        [1.0000],
        [0.2695],
        [0.3509],
        [0.0279],
        [0.0656],
        [0.0477],
        [0.0704]])}]


In [35]:
train_data[0]

HeteroData(
  user={ x=[5, 3] },
  ap={ x=[3, 3] },
  (user, uplink, ap)={ edge_attr=[15, 1] }
)

In [44]:
from torch_geometric.loader import DataLoader
batch_size = 100
train_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=1)

In [49]:
import os.path as osp

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import DBLP
from torch_geometric.nn import HGTConv, Linear

path = osp.join(osp.dirname(osp.realpath('.')), '../../data/DBLP')
# We initialize conference node features with a single one-vector as feature:
dataset = DBLP(path, transform=T.Constant(node_types='conference'))
data = dataset[0]
print(data)


Extracting D:\Giang\data\DBLP\raw\DBLP_processed.zip
Processing...


HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={
    num_nodes=20,
    x=[20, 1]
  },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
)


Done!


In [50]:
class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data.metadata(),
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict = {
            node_type: self.lin_dict[node_type](x).relu_()
            for node_type, x in x_dict.items()
        }

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return self.lin(x_dict['author'])


model = HGT(hidden_channels=64, out_channels=4, num_heads=2, num_layers=1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

with torch.no_grad():  # Initialize lazy modules.
    out = model(data.x_dict, data.edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['author'].train_mask
    loss = F.cross_entropy(out[mask], data['author'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask', 'test_mask']:
        mask = data['author'][split]
        acc = (pred[mask] == data['author'].y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs


for epoch in range(1, 101):
    loss = train()
    train_acc, val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
          f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')


Epoch: 001, Loss: 1.4033, Train: 0.3325, Val: 0.2750, Test: 0.3245
Epoch: 002, Loss: 1.3780, Train: 0.2950, Val: 0.2575, Test: 0.3012
Epoch: 003, Loss: 1.3532, Train: 0.3300, Val: 0.2725, Test: 0.3107
Epoch: 004, Loss: 1.3220, Train: 0.5775, Val: 0.4050, Test: 0.4691
Epoch: 005, Loss: 1.2793, Train: 0.5900, Val: 0.4750, Test: 0.5207
Epoch: 006, Loss: 1.2207, Train: 0.6025, Val: 0.4475, Test: 0.4980
Epoch: 007, Loss: 1.1418, Train: 0.6575, Val: 0.4475, Test: 0.5041
Epoch: 008, Loss: 1.0391, Train: 0.7450, Val: 0.4925, Test: 0.5453
Epoch: 009, Loss: 0.9117, Train: 0.8100, Val: 0.5500, Test: 0.6122
Epoch: 010, Loss: 0.7625, Train: 0.8550, Val: 0.6400, Test: 0.6712
Epoch: 011, Loss: 0.6018, Train: 0.8625, Val: 0.6650, Test: 0.7148
Epoch: 012, Loss: 0.4514, Train: 0.8625, Val: 0.6675, Test: 0.7228
Epoch: 013, Loss: 0.3324, Train: 0.8650, Val: 0.6575, Test: 0.7292
Epoch: 014, Loss: 0.2496, Train: 0.9100, Val: 0.6875, Test: 0.7482
Epoch: 015, Loss: 0.1871, Train: 0.9850, Val: 0.7200, Test: 0.

In [62]:
print(data.num_edges)

239566


In [55]:
def adj_matrix(num_aps, num_users):
    adj = []
    for i in range(num_aps):
        for j in range(num_users):
            adj.append([i, j])
    return np.array(adj)

In [57]:
adj = adj_matrix(3,5)

In [63]:
adj.transpose()

array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
       [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4]])

# Tesst Data Hetero

In [75]:
from scipy.spatial import distance_matrix
def generate_channels_wsn(num_ap, num_user, num_samples, var_noise=1.0, radius=1):
    # print("Generating Data for training and testing")

    # if num_ap != 1:
    #     raise Exception("Can not generate data for training and testing with more than 1 base station")
    # generate position
    dist_mat = []
    position = []
    index_user = np.tile(np.arange(num_user), (num_ap, 1))
    index_ap = np.tile(np.arange(num_ap).reshape(-1, 1), (1, num_user))

    index = np.array([index_user, index_ap])

    # Calculate channel
    CH = 1 / np.sqrt(2) * (np.random.randn(num_samples, 1, num_user)
                           + 1j * np.random.randn(num_samples, 1, num_user))

    if radius == 0:
        Hs = abs(CH)
    else:
        for each_sample in range(num_samples):
            pos = []
            pos_BS = []

            for i in range(num_ap):
                r = radius * (np.random.rand())
                theta = np.random.rand() * 2 * np.pi
                pos_BS.append([r * np.sin(theta), r * np.cos(theta)])
                pos.append([r * np.sin(theta), r * np.cos(theta)])
            pos_user = []

            for i in range(num_user):
                r = 0.5 * radius + 0.5 * radius * np.random.rand()
                theta = np.random.rand() * 2 * np.pi
                pos_user.append([r * np.sin(theta), r * np.cos(theta)])
                pos.append([r * np.sin(theta), r * np.cos(theta)])

            pos = np.array(pos)
            pos_BS = np.array(pos_BS)
            dist_matrix = distance_matrix(pos_BS, pos_user)
            # dist_matrixp = distance_matrix(pos[1:], pos[1:])
            dist_mat.append(dist_matrix)
            position.append(pos)

        dist_mat = np.array(dist_mat)
        position = np.array(position)

        # Calculate Free space pathloss
        # f = 2e9
        # c = 3e8
        # FSPL_old = 1 / ((4 * np.pi * f * dist_mat / c) ** 2)
        FSPL = - (120.9 + 37.6 * np.log10(dist_mat/1000))
        FSPL = 10 ** (FSPL / 10)

        # print(f'FSPL_old:{FSPL_old.sum()}')
        # print(f'FSPL_new:{FSPL.sum()}')
        Hs = abs(CH * FSPL)

    Hs, noise = normalize_matrix(Hs, var_noise)

    return Hs, noise, position, index

def channel_reshape(channel, num_ap, num_user):
    # input of (num_samples x 1)
    # output of (num_samples x num_ap x num_user)
    tmp = np.repeat(
        np.expand_dims(
            np.repeat(
                channel,
                num_ap, axis=1),
            axis=2
        ),
        num_user,
        axis=2
    )
    return tmp


def normalize_matrix(channel_matrix, noise_var):
    num_samples, num_ap, num_user = channel_matrix.shape
    max_each_sample = np.expand_dims(
        np.max(
            np.max(channel_matrix, axis=2),
            axis=-1
        ),
        axis=1
        )
    max_matrix = channel_reshape(max_each_sample, num_ap, num_user)

    min_each_sample = np.expand_dims(
        np.min(
            np.min(channel_matrix, axis=2),
            axis=-1
        ),
        axis=1
    )
    min_matrix = channel_reshape(min_each_sample, num_ap, num_user)

    noise = np.ones(channel_matrix.shape) * noise_var

    # return (
    #     (channel_matrix - min_matrix) / (max_matrix - min_matrix),
    #     (noise - min_matrix) / (max_matrix - min_matrix),
    # )
    return (
        (channel_matrix / max_matrix),
        (noise / max_matrix),
    )

In [104]:
def convert_to_hetero_data(channel_matrices):
    graph_list = []
    num_sam, num_aps, num_users = channel_matrices.shape
    for i in range(num_sam):
        user_feat = torch.randn(num_users, num_users_features)  # features of user_node
        ap_feat = torch.randn(num_aps, num_aps_features)  # features of user_node
        edge_feat_uplink = channel_matrices[i, :, :].reshape(-1, 1)
        edge_feat_downlink = channel_matrices[i, :, :].reshape(-1, 1)
        graph = HeteroData({
            'user': {'x': user_feat},
            'ap': {'x': ap_feat}
        })
        # Create edge types and building the graph connectivity:
        graph['user', 'uplink', 'ap'].edge_attr = torch.tensor(edge_feat_uplink, dtype=torch.float)
        graph['user', 'uplink', 'ap'].edge_index = torch.tensor(adj_matrix(num_users, num_aps).transpose(), dtype=torch.int64)
        graph['ap', 'downlink', 'user'].edge_index = torch.tensor(adj_matrix(num_aps, num_users).transpose(),
                                                                dtype=torch.int64)

        # graph['ap', 'downlink', 'user'].edge_attr  = torch.tensor(edge_feat_downlink, dtype=torch.float)
        graph_list.append(graph)
    return graph_list

def adj_matrix(num_from, num_dest):
    adj = []
    for i in range(num_from):
        for j in range(num_dest):
            adj.append([i, j])
    return np.array(adj)

In [105]:
K = 3  # number of APs
N = 5  # number of nodes
R = 10  # radius

num_users_features = 3
num_aps_features = 3

num_train = 2  # number of training samples
num_test = 4  # number of test samples

reg = 1e-2
pmax = 1
var_db = 10
var = 1 / 10 ** (var_db / 10)
var_noise = 10e-11

power_threshold = 2.0

X_train, noise_train, pos_train, index_train = generate_channels_wsn(K, N, num_train, var_noise, R)
X_test, noise_test, pos_test, index_test = generate_channels_wsn(K + 1, N + 10, num_test, var_noise, R)

# Maybe need normalization here
train_data = convert_to_hetero_data(X_train)
test_data = convert_to_hetero_data(X_test)

batchSize = 100
train_loader = DataLoader(train_data, batchSize, shuffle=True, num_workers=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = train_data[0]
data = data.to(device)

In [115]:
output

tensor([[-0.0613, -0.0862,  0.1738, -0.0829],
        [-0.0640, -0.0097,  0.0460, -0.0183],
        [ 0.0135, -0.1324,  0.1521, -0.1092],
        [-0.0988, -0.1225,  0.1574, -0.0855],
        [-0.0390, -0.0085,  0.0239, -0.0096]])

In [86]:
len(data['user', 'uplink', 'ap']['edge_attr'])

14

In [116]:
#region Build Heterogeneous GNN
class HetNetGNN(torch.nn.Module):
    def __init__(self, data, hidden_channels, out_channels, num_heads, num_layers):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data.metadata(),
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict = {
            node_type: self.lin_dict[node_type](x).relu_()
            for node_type, x in x_dict.items()
        }

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return self.lin(x_dict['user'])


#endregion

In [117]:
model = HetNetGNN(data, hidden_channels=64, out_channels=4, num_heads=2, num_layers=1)
model = model.to(device)

In [118]:
with torch.no_grad():
    output = model(data.x_dict, data.edge_index_dict)
print(output)
print(data)

tensor([[-0.1480, -0.1011, -0.1282, -0.0549],
        [-0.1823, -0.1279, -0.1350, -0.0248],
        [-0.1425, -0.1360, -0.0933, -0.0341],
        [-0.1568, -0.0868, -0.0459,  0.0029],
        [-0.1772, -0.0984, -0.1865, -0.0762]])
HeteroData(
  user={ x=[5, 3] },
  ap={ x=[3, 3] },
  (user, uplink, ap)={
    edge_attr=[15, 1],
    edge_index=[2, 15]
  },
  (ap, downlink, user)={ edge_index=[2, 15] }
)


## Loss function
input batch data and output of the model
output: the enegy efficieny or the sumRate

In [93]:
def loss_function(data, out, num_ap, num_user, noise_matrix, p_max, train=True, is_log=False):
    # Loss function only takes data and the output to calculate energy efficiency

    G = torch.reshape(out[:, 0], (-1, num_ap, num_user))  #/ noise
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # how to get channel from data and output
    P = torch.reshape(out[:, 2], (-1, num_ap, num_user)) * p_max
    # ## ap selection part
    # ap_select = torch.reshape(out[:, 1], (-1, num_ap, num_user))
    # P = torch.mul(P, ap_select)
    # ##
    desired_signal = torch.sum(torch.mul(P,G), dim=2).unsqueeze(-1)
    P_UE = torch.sum(P, dim=1).unsqueeze(-1)
    all_received_signal = torch.matmul(G, P_UE)
    new_noise = torch.from_numpy(noise_matrix).to(device)
    interference = all_received_signal - desired_signal + new_noise
    rate = torch.log(1 + torch.div(desired_signal, interference))
    sum_rate = torch.mean(torch.sum(rate, 1))
    mean_power = torch.mean(torch.sum(P_UE, 1))

    if is_log:
        print(f'Channel Coefficient: {G}')
        print(f'Power: {P}')
        print(f'desired_signal: {desired_signal}')
        print(f'P_UE: {P_UE}')
        print(f'all_received_signal: {all_received_signal}')
        print(f'interference: {interference}')

    if train:
        return torch.neg(sum_rate/mean_power)
    else:
        return sum_rate/mean_power

In [ ]:
def loss_function(output, batch):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    ##
    channel_matrix = batch['user', 'ap']['edge_attr']
    power = batch['user']['x'][:,0]
    G = torch.reshape(channel_matrix, (-1, num_ap, num_user))
    P = torch.reshape(power, (-1, num_ap, num_user)) * p_max
    
    ##
    desired_signal = torch.sum(torch.mul(P,G), dim=2).unsqueeze(-1)
    P_UE = torch.sum(P, dim=1).unsqueeze(-1)
    all_received_signal = torch.matmul(G, P_UE)
    new_noise = torch.from_numpy(noise_matrix).to(device)
    interference = all_received_signal - desired_signal + new_noise
    rate = torch.log(1 + torch.div(desired_signal, interference))
    sum_rate = torch.mean(torch.sum(rate, 1))
    mean_power = torch.mean(torch.sum(P_UE, 1))

In [123]:
batch['user']['x'].shape[0]

10

In [95]:
data['user', 'ap']['edge_attr']

tensor([[1.0000],
        [0.0117],
        [0.0073],
        [0.0129],
        [0.0442],
        [0.0207],
        [0.0050],
        [0.1102],
        [0.0061],
        [0.0140],
        [0.0025],
        [0.0023],
        [0.3133],
        [0.0029]])

In [100]:
batch

HeteroDataBatch(
  user={
    x=[10, 3],
    batch=[10],
    ptr=[3]
  },
  ap={
    x=[6, 3],
    batch=[6],
    ptr=[3]
  },
  (user, uplink, ap)={
    edge_attr=[28, 1],
    edge_index=[2, 30]
  },
  (ap, downlink, user)={ edge_index=[2, 30] }
)

In [131]:
power = batch['user']['x']
print(power.shape)

torch.Size([10, 3])


In [103]:
channel_matrix = batch['user', 'ap']['edge_attr']
print(channel_matrix)

tensor([[4.7116e-04],
        [6.8099e-04],
        [2.9243e-03],
        [8.0180e-04],
        [1.0000e+00],
        [3.9735e-03],
        [1.0341e-03],
        [1.4105e-01],
        [2.5859e-03],
        [5.5022e-03],
        [2.5087e-04],
        [6.4103e-03],
        [9.3652e-05],
        [1.3648e-03],
        [1.0000e+00],
        [1.1707e-02],
        [7.3273e-03],
        [1.2917e-02],
        [4.4212e-02],
        [2.0658e-02],
        [4.9644e-03],
        [1.1016e-01],
        [6.1488e-03],
        [1.3968e-02],
        [2.5157e-03],
        [2.3086e-03],
        [3.1334e-01],
        [2.9395e-03]])


In [128]:
num_user = 5
x1 = torch.ones(num_user,1)
x2 = torch.ones(num_user,1) # power allocation
x3 = torch.ones(num_user,1) # ap selection?

In [130]:
torch.cat((x1,x2,x3),1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [132]:
channel_matrix = batch['user', 'ap']['edge_attr']

power_max = batch['user']['x'][:, 0]
power = batch['user']['x'][:, 1]
ap_selection = batch['user']['x'][:, 2]


In [133]:
power_max

tensor([-1.2864, -0.4920,  0.9406,  1.1642,  0.4339, -0.6288, -0.2291,  0.8168,
        -1.0453,  0.3287])

In [143]:
power.shape

torch.Size([10])

In [142]:
ap_selection = torch.tensor([0, 1, 2, 3, 1, 2, 3, 2, 1, 0])
print(ap_selection.shape)

torch.Size([10])


In [155]:
P = torch.zeros(1, 4,10)
print(P)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])


In [151]:
for i in range(10):
    tmp = power_max[i] * power[i]
    P[0][ap_selection[i]][i] = tmp

tensor(0)
tensor(1)
tensor(2)
tensor(3)
tensor(1)
tensor(2)
tensor(3)
tensor(2)
tensor(1)
tensor(0)


In [152]:
P

tensor([[[ 0.0857,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0533],
         [ 0.0000,  0.0941,  0.0000,  0.0000,  0.1711,  0.0000,  0.0000,
           0.0000,  0.0376,  0.0000],
         [ 0.0000,  0.0000, -0.9847,  0.0000,  0.0000,  0.3065,  0.0000,
          -0.3427,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.1886,  0.0000,  0.0000, -0.0096,
           0.0000,  0.0000,  0.0000]]])

In [156]:
import torch

# Assuming power_max, power, and P are torch tensors

# Compute the element-wise product of power_max and power
tmp = power_max * power

# Create an index tensor for the first dimension of P
index = torch.arange(10)

# Update the corresponding elements in P using tensor indexing
P[0, ap_selection[index], index] = tmp

In [157]:
P

tensor([[[ 0.0857,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0533],
         [ 0.0000,  0.0941,  0.0000,  0.0000,  0.1711,  0.0000,  0.0000,
           0.0000,  0.0376,  0.0000],
         [ 0.0000,  0.0000, -0.9847,  0.0000,  0.0000,  0.3065,  0.0000,
          -0.3427,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.1886,  0.0000,  0.0000, -0.0096,
           0.0000,  0.0000,  0.0000]]])

In [158]:
index

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])